<a href="https://colab.research.google.com/github/shantanudeshp/llm-scotus/blob/20230904-wip/scotus_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Use case_data to get case details, and call the openai llm models to predict outcome.

In [ ]:
%run case_data.ipynb

Exception: ignored

In [ ]:
!pip install openai

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

In [ ]:
import os
import requests
import openai
from bs4 import BeautifulSoup

from getpass import getpass

In [ ]:
openai_key = getpass('Enter the openai key')
#openai_key = os.getenv("OPENAI_API_KEY")

Enter the openai key··········


In [ ]:
openai.organization = "org-YjWOHW4W2uUvobjJfFBOBkU8"
openai.api_key = openai_key
#openai.Model.list()

In [ ]:
instructions = '''The following is a brief outline of a case. Predict the verdict of the Supreme Court by responding with the name of the Contestant they would most likely side with.
For example:
If the contestants are Reagan v. Alabama, respond with either Reagan or Alabama.
If the contestants are Lubkowitz v. Smith, respond with either Lubkowitz or Smith.
If the contestants are Department of Defense v. Higgins, respond with either Department of Defense or Higgins.'''

In [ ]:
template1 = """
{instructions}

Case: {case_name}

Summary: {case_summary}

Verdict:
"""

print(template1)


{instructions}

Case: {case_name}

Summary: {case_summary}

Verdict:



In [ ]:
def generate(prompt, engine="text-davinci-003", max_tokens=100):
    response = openai.Completion.create(
        engine=engine,
        prompt=prompt,
        max_tokens=max_tokens
    )
    tokens_used = response.usage['total_tokens']
    return response.choices[0].text.strip(), tokens_used

In [ ]:
def get_verdict(url, engine="text-davinci-003"):
    case_details = get_case_details(url)
    # 1. Call the scrape_contestants and scrape_syllabus functions on each URL
    case_name = case_details['contestants']
    case_summary = case_details['syllabus']
    #TODO: check both are not None

    # 2. Put the outputs inside the template
    prompt = template1.format(instructions=instructions, case_name=case_name, case_summary=case_summary)

    # 3. Call the LLM API to predict the verdict for each
    predicted_verdict, tokens_used = generate(prompt, engine)

    # 4. Store the verdict and tokens used for each URL
    results = {
        "url": url,
        "predicted_verdict": predicted_verdict,
        "tokens_used": tokens_used
    }

    # 5. Return the results
    return results

In [ ]:
def get_verdicts(urls, engine="text-davinci-003"):
    verdicts = [get_verdict(u) for u in urls]
    return verdicts

In [ ]:
# Example usage:
urls = ['https://supreme.justia.com/cases/federal/us/597/21-954/']  # Add more URLs as needed
verdicts = get_verdicts(urls)

for verdict in verdicts:
    print(f"URL: {verdict['url']}, Predicted Verdict: {verdict['predicted_verdict']}, Tokens used: {verdict['tokens_used']}")

URL: https://supreme.justia.com/cases/federal/us/597/21-954/, Predicted Verdict: Biden, Tokens used: 1463
